In [81]:
import tensorflow as tf

In [82]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm  
import keras
import math
from scipy import ndimage

### Defining Image Directory

In [83]:
train_dir = "C:/Users/jaina/Downloads/hand_data/extracted_images/"

### Data : Collecting images

In [84]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

     
train_datagen = ImageDataGenerator(rescale=1./255,
    
    data_format='channels_first',
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','+','-','times','forward_slash','(',')','{','}','[',']','e','i','in','pi'],
    class_mode="sparse",
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','+','-','times','forward_slash','(',')','{','}','[',']','e','i','in','pi'],
    class_mode="sparse",
    subset='validation') 

Found 158648 images belonging to 24 classes.
Found 39650 images belonging to 24 classes.


 # Model

In [85]:
import keras
keras.backend.set_image_data_format('channels_first')

In [86]:
from keras.layers import merge, Input, concatenate
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D, Convolution2D
img_rows, img_cols = 28, 28

nb_filters_1 = 64
nb_filters_2 = 128
nb_filters_3 = 256
nb_conv = 3
nb_conv_mid = 4
nb_conv_init = 5

init = Input(shape=(1, 28, 28),)

fork11 = Convolution2D(nb_filters_1, nb_conv_init, nb_conv_init,  activation="relu", border_mode='same')(init)
fork12 = Convolution2D(nb_filters_1, nb_conv_init, nb_conv_init, activation="relu", border_mode='same')(init)
merge1 = concatenate([fork11, fork12], axis=1, name='merge1')
# concat_feat = concatenate([concat_feat, x], mode='concat', axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
maxpool1 = MaxPooling2D(strides=(2,2), border_mode='same')(merge1)

fork21 = Convolution2D(nb_filters_2, nb_conv_mid, nb_conv_mid, activation="relu", border_mode='same')(maxpool1)
fork22 = Convolution2D(nb_filters_2, nb_conv_mid, nb_conv_mid, activation="relu", border_mode='same')(maxpool1)
merge2 = concatenate([fork21, fork22, ], axis=1, name='merge2')
maxpool2 = MaxPooling2D(strides=(2,2), border_mode='same')(merge2)

fork31 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork32 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork33 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork34 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork35 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork36 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
merge3 = concatenate([fork31, fork32, fork33, fork34, fork35, fork36, ], axis=1, name='merge3')
maxpool3 = MaxPooling2D(strides=(2,2), border_mode='same')(merge3)
maxpool4 = MaxPooling2D(strides=(2,2), border_mode='same')(maxpool3)
flatten = Flatten()(maxpool4)
dense1 = Dense(96, activation="relu")(flatten)
dense2 = Dense(62, activation="relu")(dense1)
dense3 = Dense(48, activation="relu")(dense2)
output = Dense(24, activation="softmax")(dense3)

model = Model(input=init, output=output)

model.summary()

C:\Users\jaina\anaconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", padding="same")`
  app.launch_new_instance()
C:\Users\jaina\anaconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", padding="same")`
C:\Users\jaina\anaconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(strides=(2, 2), padding="same")`
C:\Users\jaina\anaconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (4, 4), activation="relu", padding="same")`
C:\Users\jaina\anaconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (4, 4), activation="relu", padding="same")`
C:\Users

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 1, 28, 28)    0                                            
__________________________________________________________________________________________________
conv2d_101 (Conv2D)             (None, 64, 28, 28)   1664        input_11[0][0]                   
__________________________________________________________________________________________________
conv2d_102 (Conv2D)             (None, 64, 28, 28)   1664        input_11[0][0]                   
__________________________________________________________________________________________________
merge1 (Concatenate)            (None, 128, 28, 28)  0           conv2d_101[0][0]                 
                                                                 conv2d_102[0][0]          

In [87]:
from keras import optimizers
ada = keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)
model.compile(optimizer=ada,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              validation_steps=100,
                              epochs=50)

Epoch 1/50
100/100 [==============================] - 6s 62ms/step - loss: 2.8019 - accuracy: 0.1345 - val_loss: 3.0511 - val_accuracy: 0.1745
Epoch 2/50
100/100 [==============================] - 5s 53ms/step - loss: 2.2330 - accuracy: 0.3265 - val_loss: 2.1080 - val_accuracy: 0.3945
Epoch 3/50
100/100 [==============================] - 5s 53ms/step - loss: 1.7254 - accuracy: 0.4690 - val_loss: 1.8559 - val_accuracy: 0.6590
Epoch 4/50
100/100 [==============================] - 5s 54ms/step - loss: 1.1923 - accuracy: 0.6770 - val_loss: 0.4657 - val_accuracy: 0.7360
Epoch 5/50
100/100 [==============================] - 5s 54ms/step - loss: 0.9394 - accuracy: 0.7335 - val_loss: 0.3871 - val_accuracy: 0.7660
Epoch 6/50
100/100 [==============================] - 5s 55ms/step - loss: 0.7260 - accuracy: 0.7835 - val_loss: 0.4332 - val_accuracy: 0.8150
Epoch 7/50
100/100 [==============================] - 5s 54ms/step - loss: 0.5789 - accuracy: 0.8365 - val_loss: 0.5459 - val_accuracy: 0.8480

In [88]:
model.save('C:/Users/jaina/Downloads/hand_data/DCNN_10AD_sy_better_pool_without_drop.h5')